In [39]:
import nltk
import pyarrow as pa
import pyarrow.parquet as pq
import sklearn
from sklearn.preprocessing import StandardScaler
import seaborn as sns
    
from sklearn.linear_model import LinearRegression

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd

In [40]:
df = pd.read_parquet('gs://data_henry_proyecto_final/gmap_ML.parquet')

In [41]:
df

,user_id,rating,text,name_y,latitude,longitude,category,avg_rating,num_of_reviews,Estado
0,1.089912e+20,5,Love there korean rice cake.,San Soo Dang,34.058092,-118.292130,Korean restaurant,4.4,18,CA
1,1.089912e+20,5,Love there korean rice cake.,San Soo Dang,34.058092,-118.292130,Korean restaurant,4.4,18,CA
2,1.112903e+20,5,Good very good,San Soo Dang,34.058092,-118.292130,Korean restaurant,4.4,18,CA
3,1.112903e+20,5,Good very good,San Soo Dang,34.058092,-118.292130,Korean restaurant,4.4,18,CA
4,1.126404e+20,4,They make Korean traditional food very properly.,San Soo Dang,34.058092,-118.292130,Korean restaurant,4.4,18,CA
...,...,...,...,...,...,...,...,...,...,...
3549522,1.115437e+20,5,Great staff and love the hunts and getting out...,74 Ranch Resort,28.677744,-98.373761,Resort hotel,5.0,28,TX
3549523,1.109932e+20,5,Great accommodations and a blast,74 Ranch Resort,28.677744,-98.373761,Resort hotel,5.0,28,TX
3549524,1.168874e+20,5,This place is awesome. Had a great quail hunt....,74 Ranch Resort,28.677744,-98.373761,Resort hotel,5.0,28,TX
3549525,1.144327e+20,5,This is the place to go if you want a first cl...,74 Ranch Resort,28.677744,-98.373761,Resort hotel,5.0,28,TX


In [42]:
df = df.drop('user_id',axis=1)

In [45]:
df = df.drop('Estado',axis=1)

In [6]:
df = df.drop('avg_rating',axis=1)
df = df.drop('category',axis=1)
df = df.drop('num_of_reviews',axis=1)
df = df.drop('latitude',axis=1)
df = df.drop('longitude',axis=1)
df = df.drop('rating',axis=1)

In [43]:
df = df.drop_duplicates()

In [46]:
df

,rating,text,name_y,latitude,longitude,category,avg_rating,num_of_reviews
0,5,Love there korean rice cake.,San Soo Dang,34.058092,-118.292130,Korean restaurant,4.4,18
2,5,Good very good,San Soo Dang,34.058092,-118.292130,Korean restaurant,4.4,18
4,4,They make Korean traditional food very properly.,San Soo Dang,34.058092,-118.292130,Korean restaurant,4.4,18
6,5,Short ribs are very delicious.,San Soo Dang,34.058092,-118.292130,Korean restaurant,4.4,18
8,5,Great food and prices the portions are large,San Soo Dang,34.058092,-118.292130,Korean restaurant,4.4,18
...,...,...,...,...,...,...,...,...
3549522,5,Great staff and love the hunts and getting out...,74 Ranch Resort,28.677744,-98.373761,Resort hotel,5.0,28
3549523,5,Great accommodations and a blast,74 Ranch Resort,28.677744,-98.373761,Resort hotel,5.0,28
3549524,5,This place is awesome. Had a great quail hunt....,74 Ranch Resort,28.677744,-98.373761,Resort hotel,5.0,28
3549525,5,This is the place to go if you want a first cl...,74 Ranch Resort,28.677744,-98.373761,Resort hotel,5.0,28


In [47]:
comentarios = df['text']

In [48]:
analyzer = SentimentIntensityAnalyzer()

In [24]:
def analyze_sentiment(text):
    scores = analyzer.polarity_scores(text)
    # Asume que 'compound' es el puntaje de sentimiento compuesto
    if scores['compound'] >= 0.05:
        return 1  # Positivo
    else:
        return 0  # Negativo
parts = np.array_split(df, 100)
results = []

for part in parts:
    result = part['text'].apply(analyze_sentiment)
    results.append(result)

final_result = pd.concat(results)


/opt/conda/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [49]:
final_result

0          1
2          1
4          0
6          1
8          1
          ..
3549522    1
3549523    1
3549524    1
3549525    1
3549526    1
Name: text, Length: 1331735, dtype: int64

In [59]:
# Asume que 'df' es tu DataFrame original y 'final_result' es el resultado del análisis de sentimientos
df_with_sentiments = pd.concat([df, final_result], axis=1)

# Si deseas cambiar el nombre de la columna de sentimientos puedes hacerlo así:
df_with_sentiments = df_with_sentiments.rename(columns={0: 'sentiment'})


In [62]:
df_with_sentiments

,rating,name_y,latitude,longitude,category,avg_rating,num_of_reviews,text
0,5,San Soo Dang,34.058092,-118.292130,Korean restaurant,4.4,18,1
2,5,San Soo Dang,34.058092,-118.292130,Korean restaurant,4.4,18,1
4,4,San Soo Dang,34.058092,-118.292130,Korean restaurant,4.4,18,0
6,5,San Soo Dang,34.058092,-118.292130,Korean restaurant,4.4,18,1
8,5,San Soo Dang,34.058092,-118.292130,Korean restaurant,4.4,18,1
...,...,...,...,...,...,...,...,...
3549522,5,74 Ranch Resort,28.677744,-98.373761,Resort hotel,5.0,28,1
3549523,5,74 Ranch Resort,28.677744,-98.373761,Resort hotel,5.0,28,1
3549524,5,74 Ranch Resort,28.677744,-98.373761,Resort hotel,5.0,28,1
3549525,5,74 Ranch Resort,28.677744,-98.373761,Resort hotel,5.0,28,1


In [52]:
# Asume que 'df' es tu DataFrame y 'new_name' es el nuevo nombre que quieres darle a la columna
new_name = 'borrar'
df.columns.values[1] = new_name  # Recuerda que Python usa indexación basada en 0


In [55]:
df

,rating,borrar,name_y,latitude,longitude,category,avg_rating,num_of_reviews
0,5,Love there korean rice cake.,San Soo Dang,34.058092,-118.292130,Korean restaurant,4.4,18
2,5,Good very good,San Soo Dang,34.058092,-118.292130,Korean restaurant,4.4,18
4,4,They make Korean traditional food very properly.,San Soo Dang,34.058092,-118.292130,Korean restaurant,4.4,18
6,5,Short ribs are very delicious.,San Soo Dang,34.058092,-118.292130,Korean restaurant,4.4,18
8,5,Great food and prices the portions are large,San Soo Dang,34.058092,-118.292130,Korean restaurant,4.4,18
...,...,...,...,...,...,...,...,...
3549522,5,Great staff and love the hunts and getting out...,74 Ranch Resort,28.677744,-98.373761,Resort hotel,5.0,28
3549523,5,Great accommodations and a blast,74 Ranch Resort,28.677744,-98.373761,Resort hotel,5.0,28
3549524,5,This place is awesome. Had a great quail hunt....,74 Ranch Resort,28.677744,-98.373761,Resort hotel,5.0,28
3549525,5,This is the place to go if you want a first cl...,74 Ranch Resort,28.677744,-98.373761,Resort hotel,5.0,28


In [32]:
columnas = df.columns.tolist()

columnas

In [33]:
columnas

['rating',
 'borrar',
 'name_y',
 'latitude',
 'longitude',
 'category',
 'avg_rating',
 'num_of_reviews']

In [56]:
df.columns = df.columns.str.strip()


In [57]:
df = df.drop('borrar',axis=1)

In [63]:
df_with_sentiments

,rating,name_y,latitude,longitude,category,avg_rating,num_of_reviews,text
0,5,San Soo Dang,34.058092,-118.292130,Korean restaurant,4.4,18,1
2,5,San Soo Dang,34.058092,-118.292130,Korean restaurant,4.4,18,1
4,4,San Soo Dang,34.058092,-118.292130,Korean restaurant,4.4,18,0
6,5,San Soo Dang,34.058092,-118.292130,Korean restaurant,4.4,18,1
8,5,San Soo Dang,34.058092,-118.292130,Korean restaurant,4.4,18,1
...,...,...,...,...,...,...,...,...
3549522,5,74 Ranch Resort,28.677744,-98.373761,Resort hotel,5.0,28,1
3549523,5,74 Ranch Resort,28.677744,-98.373761,Resort hotel,5.0,28,1
3549524,5,74 Ranch Resort,28.677744,-98.373761,Resort hotel,5.0,28,1
3549525,5,74 Ranch Resort,28.677744,-98.373761,Resort hotel,5.0,28,1


In [69]:
# Asume que 'df' es tu DataFrame
df_agrupado = df_with_sentiments.groupby('name_y')['text'].sum().reset_index()


In [73]:
df_agrupado

,name_y,text
0,"""TACOS"" MICHOACAN",26
1,#8:46,26
2,$,35
3,'Essen,30
4,.,19
...,...,...
19252,马师傅面馆,3
19253,魔饭 Morefan,8
19254,옛날통닭 & 스시 | Chicken and Sushi Flushing NY,29
19255,초원가든 Chowon Garden,7


In [74]:
# Asumiendo que 'df' es tu DataFrame original y 'df_agrupado' es el DataFrame agrupado
columnas = ['name_y', 'latitude', 'longitude', 'num_of_reviews','avg_rating']
df_merge = pd.merge(df_agrupado, df[columnas].drop_duplicates(), on='name_y', how='left')



In [75]:
df_merge

,name_y,text,latitude,longitude,num_of_reviews,avg_rating
0,"""TACOS"" MICHOACAN",26,34.193687,-118.531962,88,4.1
1,#8:46,26,30.057229,-95.494884,57,4.4
2,$,35,26.348095,-80.084331,158,3.3
3,'Essen,30,40.759743,-73.973728,118,4.1
4,.,19,40.111530,-75.336822,56,4.2
...,...,...,...,...,...,...
23094,马师傅面馆,3,32.951999,-96.727767,38,3.9
23095,魔饭 Morefan,8,33.067249,-96.696350,28,4.0
23096,옛날통닭 & 스시 | Chicken and Sushi Flushing NY,29,40.761944,-73.803939,124,4.2
23097,초원가든 Chowon Garden,7,40.763264,-73.808759,35,3.8


In [77]:
import pyarrow as pa
import pyarrow.parquet as pq

In [79]:
table = pa.Table.from_pandas(df_merge)

In [80]:
schema = table.schema

In [81]:
with pq.ParquetWriter('gs://data_henry_proyecto_final/google_analisis_sentimen.parquet', schema) as writer:
    writer.write_table(table)

In [82]:
def calcular_puntaje(fila):
    puntaje = 0
    
    pesos = {'text': -2, 'avg_rating': 1, 'num_of_reviews': 1, 'num_of_reviews': 1, }
    for columna, peso in pesos.items():
        puntaje += peso * fila[columna]
    return puntaje

In [85]:
df_merge['puntaje'] = df_merge.apply(calcular_puntaje, axis=1)